In [1]:
import numpy  as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img
import os
import pandas as pd
import pathlib
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [2]:
#data_dir = pathlib.Path('./German-Traffic-Sign/Meta')
train_path = pathlib.Path('./German-Traffic-Sign/Train')
test_path = pathlib.Path('./German-Traffic-Sign/Test')
NUM_CATEGORIES = len(os.listdir(train_path))

def load_data(data_dir):
    images = list()
    labels = list()
    for category in range(NUM_CATEGORIES):
        categories = os.path.join(data_dir, str(category))
        for img in os.listdir(categories):
            img = load_img(os.path.join(categories, img), target_size=(30, 30))
            image = img_to_array(img) # 이미지를 넘파이 배열로 변환
            images.append(image) 
            
    
    return images



In [3]:
# 이미지 셋
images = load_data("./German-Traffic-Sign/Train/")

# 타겟 셋
target = []
for j in range(43):
    path = './German-Traffic-Sign/Train/{}'.format(j)
    polder = os.listdir(path)
    for i in polder :
        target.append(j)
target = np.array(target)

# 정규화
images = np.array(images)
images = images / 255.0

In [18]:
# 패션데이터 읽어들이기
(train_input, train_target),(test_input, test_target) = keras.datasets.fashion_mnist.load_data()


In [19]:
print(train_input.shape,train_target.shape)
print(test_input.shape,test_target.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [20]:
train_scaled = train_input.reshape(-1,28,28,1) / 255.0
train_scaled.shape

(60000, 28, 28, 1)

In [21]:
train_scaled = train_input.reshape(-1,28,28,1) / 255.0
train_scaled.shape

(60000, 28, 28, 1)

In [22]:
# 훈련 및 검증 데이터셋으로 분류하기(8:2)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled,train_target,
                                                                      test_size=0.2,random_state=42)
print(train_scaled.shape,train_target.shape)
print(val_scaled.shape,val_target.shape)

(48000, 28, 28, 1) (48000,)
(12000, 28, 28, 1) (12000,)


In [23]:
from keras.models import Sequential
from tensorflow import keras
from sklearn.linear_model import SGDClassifier
from keras.regularizers import l2
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense,Activation, MaxPool2D, BatchNormalization, Dropout
import tensorflow as tf
model = Sequential()

# 첫 번째 층 (CONV + POOL + 배치 정규화)
model.add(Conv2D(filters = 64, kernel_size=(3,3), strides=(1,1), padding='valid',input_shape=(28,28,1)))
model.add(Activation("relu"))

model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(BatchNormalization())

# 두 번째 층 (CONV + POOL + 배치 정규화)
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1),padding="same", kernel_regularizer=l2(0.0005)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))
model.add(BatchNormalization())

# 세번째 층 (CONV + 배치 정규화)
# - AlexNet 논문에서는 이 자리에 풀링층을 배치하지 않았다.
model.add(Conv2D(filters=384, kernel_size=(3,3),strides=(1,1), padding="same", kernel_regularizer=l2(0.0005)))
model.add(Activation("relu"))
model.add(BatchNormalization())

# 네 번째 층 (CONV + 배치 정규화)
# - 세 번째 층과 같은 구조
model.add(Conv2D(filters=384, kernel_size=(3,3),strides=(1,1), padding="same", kernel_regularizer=l2(0.0005)))
model.add(Activation("relu"))
model.add(BatchNormalization())

# 다섯 번째 층 (CONV + 배치 정규화)
model.add(Conv2D(filters=256, kernel_size=(3,3),strides=(1,1), padding="same", kernel_regularizer=l2(0.0005)))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid"))

# - CNN의 출력을 1차원으로 변환해 전결합층에 입력한다.
model.add(Flatten())

# 여섯 번째 층 (FC + 드롭아웃)
model.add(Dense(units = 4096, activation = 'relu'))
model.add(Dropout(0.5))

# 일곱 번째 층 (FC + 드롭아웃)
model.add(Dense(units = 4096, activation = 'relu'))
model.add(Dropout(0.5))

# 여덟 번째 층 (FC(소프트맥스 함수))
model.add(Dense(units = 10, activation = 'softmax'))

# 모델의 개요를 출력한다.
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 activation_5 (Activation)   (None, 26, 26, 64)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 batch_normalization_5 (Batc  (None, 12, 12, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, 12, 12, 256)       409856    
                                                                 
 activation_6 (Activation)   (None, 12, 12, 256)      

In [24]:
from keras.callbacks import ReduceLROnPlateau

### 검증 오차가 정체될 때마다 학숩률을 1/10로 감소한다
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1))

### SGD 옵티마이저를 학습률 0.01, 모멘텀 0.9로 설정한다
optimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)



In [27]:
### 모델을 컴파일한다
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,
              metrics=['accuracy'])

### 모델을 학습하고 콜백함수로 설정된 reduce_lr을 이용해 학습률을 감소한다
history  = model.fit(train_scaled, train_target, batch_size=128, epochs=90,
          validation_data=(val_scaled, val_target), verbose=1, callbacks=[reduce_lr])



Epoch 1/90
 69/375 [====>.........................] - ETA: 4:08 - loss: 1.7285 - accuracy: 0.6414

KeyboardInterrupt: 